### Required Libraries

In [0]:
from pyspark.sql.functions import to_date, col, trim, regexp_replace, count, sum, round, rank, initcap, split, collect_list
from pyspark.sql.functions import col, when, to_date, split, length
from pyspark.sql.functions import col, lpad, concat_ws, split, to_date
from pyspark.sql.window import Window


### Data Type Casting

In [0]:
raw_police_shooting_df = spark.table("fatal_police_shootings_data_csv")
raw_police_shooting_df = raw_police_shooting_df.withColumn("year", split(col("date"), "/")[2])
raw_police_shooting_df = raw_police_shooting_df.filter(col("year").isNotNull())

police_shooting_df = raw_police_shooting_df.withColumn("id", raw_police_shooting_df["id"].cast("integer"))
police_shooting_df = police_shooting_df.withColumn("age", police_shooting_df["age"].cast("integer"))
police_shooting_df = raw_police_shooting_df.withColumn("year", raw_police_shooting_df["year"].cast("integer"))
police_shooting_df = police_shooting_df.withColumn("longitude", police_shooting_df["longitude"].cast("float"))
police_shooting_df = police_shooting_df.withColumn("latitude", police_shooting_df["latitude"].cast("float"))

display(police_shooting_df.head(5))

id,date,threat_type,flee_status,armed_with,city,county,state,latitude,longitude,location_precision,name,age,gender,race,race_source,was_mental_illness_related,body_camera,agency_ids,year
3,2/1/2015,point,not,gun,Shelton,Mason,WA,47.246826171875,-123.12158966064453,not_available,Tim Elliot,53,male,A,not_available,TRUE,FALSE,73,2015
4,2/1/2015,point,not,gun,Aloha,Washington,OR,45.487422943115234,-122.89169311523438,not_available,Lewis Lee Lembke,47,male,W,not_available,FALSE,FALSE,70,2015
5,3/1/2015,move,not,unarmed,Wichita,Sedgwick,KS,37.694766998291016,-97.28055572509766,not_available,John Paul Quintero,23,male,H,not_available,FALSE,FALSE,238,2015
8,4/1/2015,point,not,replica,San Francisco,San Francisco,CA,37.762908935546875,-122.42200469970703,not_available,Matthew Hoffman,32,male,W,not_available,TRUE,FALSE,196,2015
9,4/1/2015,point,not,other,Evans,Weld,CO,40.38393783569336,-104.6922607421875,not_available,Michael Rodriguez,39,male,H,not_available,FALSE,FALSE,473,2015


### Split Filled and Empty counties data

In [0]:
missed_county_df = police_shooting_df.filter(police_shooting_df["county"].isNull())
filled_county_df = police_shooting_df.filter(police_shooting_df["county"].isNotNull())


### Write Filled and Empty counties to delta tables

In [0]:
filled_county_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("default.police_shooting_data_filled_county_default_data")
missed_county_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("default.police_shooting_missing_county_default_data")
